# widget test

In [4]:
import json
import matplotlib.pyplot as plt
import base64
from io import BytesIO
from IPython.display import display, HTML
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import base64
from io import BytesIO
from IPython.display import display, HTML
import torch as t

In [226]:
def to_b64_img(activation, width=10):
    size = activation.shape[0]
    assert size % width == 0
    activation = activation.detach().cpu()

    activation = activation.view(-1, 10)
    plt.figure(figsize=(2, 2))
    plt.imshow(activation)
    plt.axis('off')
    plt.tight_layout()
    buf = BytesIO()
    plt.savefig(buf, format='png')
    plt.close()
    return base64.b64encode(buf.getvalue()).decode('utf-8')

a = t.rand(100)
to_b64_img(a)

'iVBORw0KGgoAAAANSUhEUgAAAMgAAADICAYAAACtWK6eAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/H5lhTAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAE2ElEQVR4nO3dTYiVZRzG4TM6fuRMTmXZhB+Z1UhkJRhBUZlpLcRFiRhFkIQbi5IIIloUUZhBKC4mCUpbmNRABAUx4Vgmg4IU1iKisSwttCk1BWdCTU67VnEvzn9RyHXtn9sHjr95Vy9vW7PZbDaAfzXmv74A/J8JBAKBQCAQCAQCgUAgEAgEAoFAIBAIBAKBQCAQCAQCgUAgaK8cfmTvo+UL9E7fUd54/JdF5Y3dA3PLG1+u3FDeWLHwofLGweXd5Y2p+86WN0a6S/+9Go1Go7F37abyxpju/a2fLf/rcB4TCAQCgUAgEAgEAoFAIBAIBAKBQCAQCAQCgUAgEAgEAoFAIBAIBAJB6Y2WXXuuL1/g4P2flDcGd9VfdprV/2d5Y9XCJeWNddu3lTc+H5lT3vh447TyxvBT88ob/aMTyhuVX8UTBAKBQCAQCAQCgUAgEAgEAoFAIBAIBAKBQCAQCAQCgUAgEAgEAoFAIBC0NZvNZquH3x66tXyBF3bfV96YfMlIeWNSX1d5Y2TFyfJGz5TfyxtDxy4rb1w48XR5Y/vcd8sby+cvLW/0H+lt+awnCAQCgUAgEAgEAoFAIBAIBAKBQCAQCAQCgUAgEAgEAoFAIBAIBAKBQCAofWHqnZX1Lyrd9OrP5Y3j668sbxy+ozzRmLGpo7wx+tWx+j0mnipvnJl5aXljYPNF5Y1zw7+VNyo8QSAQCAQCgUAgEAgEAoFAIBAIBAKBQCAQCAQCgUAgEAgEAoFAIBAIBAJB6YWpw3d2li+wY/am8saWl+eVNz568e7yxgV7hsobjbFjyxPfPje7vPHk4v7yxvi2c+W

In [308]:
def norm_saliencies(saliencies):
    maxi = max([s for saliency in saliencies for s in saliency])
    mini = min([s for saliency in saliencies for s in saliency])
    scale = max(maxi, abs(mini))
    return [[s / scale for s in saliency] for saliency in saliencies]

def get_colors(tokens, idx):
    return ['red' if i < idx else 'blue' for i in range(len(tokens))]

def to_ch(v):
    return itoa[v] if itoa[v] != '\r' else '\\\\r' # this break my json somehow

def get_probs(out, token, k=5):
    probs = out.softmax(dim=-1)
    tops = probs.topk(k)
    pairs = [[to_ch(i.item()), v.item()] for i, v in zip(tops.indices, tops.values)]
    if token not in [p for p, _ in pairs]:
        pairs += [['...', 0.], [token, probs[atoi[token]].item()]]
    return pairs

In [309]:
def get_db(tokens):
    db = {}
    default_img = to_b64_img(t.zeros(100))
    # saliencies = norm_saliencies(stacked_lstm.saliency(tokens))
    outs, activations = stacked_lstm.stats(tokens)
    outs, activations = outs[0], activations[0]
    # activations = t.rand((len(tokens), 100))
    db['default_image'] = default_img
    db['tokens'] = list(tokens)
    # db['colors'] = [get_colors(tokens, i) for i in range(len(tokens))]
    db['colors'] = norm_saliencies(stacked_lstm.saliency(tokens))
    db['images'] = [default_img] + [to_b64_img(activation) for activation in activations]
    db['probs'] = [[]] + [get_probs(out, token) for out, token in zip(outs, tokens[1:])]
    return json.dumps(db)

# get_db('Alice in wonderland')

In [310]:
template = '''
<style>
.container {
    display: flex;
    width: 100%;
    gap: 50px;
    align-items: flex-start;
}
.infos {
    flex: 0 0 auto;
    display: flex;
    flex-direction: column;
    gap: 20px;
}
.current {
    background-color: green;
}
.prob {
    display: flex;
    gap: 20px;
}
.activations {
}
.probs .token {
    min-width: 8px;
    min-height: 16px;
}
.tokens {
    flex: 1 1 auto;
    display: flex;
    flex-wrap: wrap;
    gap: 10px;
}
.token {
    border: 1px solid grey;
    padding: 3px 10px 3px 10px;
}
</style>
    
<div class="container">
    <div class="infos">
        <div class="activations"><img /></div>
        <div class="probs"></div>
    </div>
    <div class="tokens"></div>
</div>
'''

# js_template = '''
# <script>
js_template = '''
function handle_mouseover(event) {
    el = event.target
    idx = el.getAttribute('data-idx');
    // image of the weights
    document.querySelector('.activations img').src = "data:image/png;base64," + db2['images'][idx];
    // probability for the top tokens
    probs = document.querySelector('.infos .probs');
    db2['probs'][idx].forEach((vals) => {
        div = document.createElement('div');
        div.classList.add('prob');
        prob = document.createElement('div');
        prob.textContent = vals[1].toFixed(2);
        token = document.createElement('div');
        token.classList.add('token');
        if (vals[0] === db2['tokens'][idx]) { token.classList.add('current'); }
        token.textContent = vals[0]
        div.appendChild(prob);
        div.appendChild(token);
        probs.appendChild(div);
    });
    // saliency maps for tokens
    document.querySelectorAll('.tokens .token').forEach((el, i) => {
        if (i < idx) {
            value = db2['colors'][idx][i];
            if (value < 0) {
                const t = (value + 1) / 1;
                red = Math.round(255 * t);
                green = Math.round(255 * t);
                blue = Math.round(255 * (1 - t) + 255 * t);
            } else {
                const t = value / 1;
                red = Math.round(255);
                green = Math.round(255 * (1 - t));
                blue = Math.round(255 * (1 - t));
            }
            el.style.backgroundColor = `rgb(${red}, ${green}, ${blue})`;
        }
    });
}

function handle_mouseout(event) {
    document.querySelector('.infos .activations img').src = "data:image/png;base64," + db2['default_image'];
    document.querySelector('.infos .probs').innerHTML = '';
    document.querySelectorAll('.token').forEach((el, i) => {
        el.style.backgroundColor = '';
    });
}

function start() {
    el = document.querySelector('.container .tokens');
    db2['tokens'].forEach((token, idx) => {
        span = document.createElement('span');
        span.textContent = token;
        span.setAttribute('class', 'token');
        span.setAttribute('data-idx', idx);
        span.addEventListener('mouseover', handle_mouseover);
        span.addEventListener('mouseout', handle_mouseout);
        el.appendChild(span);
    });
    document.querySelector('.activations img').src = "data:image/png;base64," + db2['default_image'];
}

start();
</script>
'''

In [312]:
# db_json = get_db('Alice in like Alice')
db_json = get_db('And so the White Rabbit butteres!')
db_template = f"<script>var db2 = JSON.parse('{db_json}');"
display(HTML(template + db_template + js_template))

In [ ]:
print(template + db_template + js_template)

# saliency

In [190]:
import torch.nn as nn
import torch.nn.functional as F
import requests

device = 'cpu'

atoi = t.load('weights/atoi.pt')
itoa = t.load('weights/itoa.pt')
d_vocab = len(atoi)
d_hidden = 100
d_batch = 10000

In [224]:
class LSTMCell(nn.Module):
    def __init__(self, d_in, d_hidden):
        super().__init__()
        self.d_hidden = d_hidden
        self.W_f = nn.Linear(d_in + d_hidden, d_hidden)  # forget gate
        self.W_i = nn.Linear(d_in + d_hidden, d_hidden)  # input gate
        self.W_c = nn.Linear(d_in + d_hidden, d_hidden)  # cell state update
        self.W_o = nn.Linear(d_in + d_hidden, d_hidden)  # output gate

    def forward(self, x, h_prev, c_prev):
        x = t.cat((x, h_prev), dim=1)
        # handle long-term memory `C`
        f_gate = t.sigmoid(self.W_f(x))
        i_gate = t.sigmoid(self.W_i(x))
        c_update = t.tanh(self.W_c(x))
        c_prev = f_gate * c_prev + i_gate * c_update
        # handle short-term memory `h`
        o_gate = t.sigmoid(self.W_o(x))
        h_prev = o_gate * t.tanh(c_prev)
        return h_prev, c_prev

class StackedLSTM(nn.Module):
    def __init__(self, d_in, d_hidden, d_out, d_layers):
        super().__init__()
        self.d_hidden = d_hidden
        self.d_layers = d_layers
        self.lstm_cells = nn.ModuleList([LSTMCell(d_in if l == 0 else d_hidden, d_hidden) for l in range(d_layers)])
        self.unembed = nn.Linear(d_hidden, d_out)

    def forward(self, xs, h_prev=None, c_prev=None):
        # xs: (batch, d_context, d_vocab)
        batch, d_context, _ = xs.shape
        outs = []
        if h_prev is None: h_prev = t.zeros(self.d_layers, batch, self.d_hidden, device=xs.device)
        if c_prev is None: c_prev = t.zeros(self.d_layers, batch, self.d_hidden, device=xs.device)
        for i in range(d_context):
            x = xs[:, i]
            h_next, c_next = [], []
            for lstm_cell, h, c in zip(self.lstm_cells, h_prev, c_prev):
                h, c = lstm_cell(x, h, c)
                h_next.append(h)
                c_next.append(c)
                x = h
            outs.append(self.unembed(h))
            h_prev = t.stack(h_next)
            c_prev = t.stack(c_next)
        return t.stack(outs, dim=1)

    @t.no_grad()
    def sample(self, seed='A', d_sample=100):
        text = seed
        h_prev = t.zeros(self.d_layers, 1, self.d_hidden, device=device)
        c_prev = t.zeros(self.d_layers, 1, self.d_hidden, device=device)
        while len(text) < d_sample:
            x = F.one_hot(t.tensor([atoi[text[-1]]]), num_classes=d_vocab).float().to(device)
            h_next, c_next = [], []
            for lstm_cell, h, c in zip(self.lstm_cells, h_prev, c_prev):
                h, c = lstm_cell(x, h, c)
                h_next.append(h)
                c_next.append(c)
                x = h
            h_prev = t.stack(h_next)
            c_prev = t.stack(c_next)
            out = self.unembed(h)
            probs = out[0].softmax(-1)
            next_sample = t.multinomial(probs, num_samples=1)
            text += itoa[next_sample.item()]
        return text

    def saliency(self, text):
        xs = F.one_hot(t.tensor([[atoi[c] for c in text]]), num_classes=d_vocab).float().to(device)
        xs.requires_grad = True
        _, d_context, _ = xs.shape
        outs = []
        h_prev = t.zeros(self.d_layers, 1, self.d_hidden, device=device)
        c_prev = t.zeros(self.d_layers, 1, self.d_hidden, device=device)
        for i in range(d_context):
            x = xs[:, i]
            h_next, c_next = [], []
            for lstm_cell, h, c in zip(self.lstm_cells, h_prev, c_prev):
                h, c = lstm_cell(x, h, c)
                h_next.append(h)
                c_next.append(c)
                x = h
            outs.append(self.unembed(h))
            h_prev = t.stack(h_next)
            c_prev = t.stack(c_next)
        saliencies = [()] # nothing caused the first token
        for idx, (out, tok) in enumerate(zip(outs, text[1:])):
            tok_idx = atoi[tok]
            # print(f'{out.shape=}')
            loss = -out[0, tok_idx]
            xs.grad = None
            loss.backward(retain_graph=True)
            # print('__')
            grads = []
            for i in range(idx + 1):
                inp_idx = atoi[text[i]]
                grads.append(xs.grad[0, i, inp_idx].item())
                # print(f'{xs.grad[0, i, inp_idx]=}')
            saliencies.append(grads)
        return saliencies

    def stats(self, text):
        # xs: (batch, d_context, d_vocab)
        xs = F.one_hot(t.tensor([[atoi[c] for c in text]]), num_classes=d_vocab).float().to(device)
        batch, d_context, _ = xs.shape
        outs = []
        hs = []
        h_prev = t.zeros(self.d_layers, batch, self.d_hidden, device=xs.device)
        c_prev = t.zeros(self.d_layers, batch, self.d_hidden, device=xs.device)
        for i in range(d_context):
            x = xs[:, i]
            h_next, c_next = [], []
            for lstm_cell, h, c in zip(self.lstm_cells, h_prev, c_prev):
                h, c = lstm_cell(x, h, c)
                h_next.append(h)
                c_next.append(c)
                x = h
            hs.append(h)
            outs.append(self.unembed(h))
            h_prev = t.stack(h_next)
            c_prev = t.stack(c_next)
        return t.stack(outs, dim=1), t.stack(hs, dim=1)

stacked_lstm = StackedLSTM(d_vocab, d_hidden, d_vocab, 3).to(device)
# stacked_lstm.sample()
# stacked_lstm.saliency('Once upon a time in a land far away') 
stacked_lstm.load_state_dict(t.load('weights/stacked_lstm_3_layers_long_2024-06-08_17h37.pt', map_location=device))
stacked_lstm.sample()

'Alice;\r\nbut it to be true_ the several other guess theyve\r\nday about the\r\ndistaking had been, and no'